In [ ]:
import os
import json
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def json_load(fname):    
    with open(fname, 'r') as f:
        return json.load(f)      

# get logs
exp_name = "t1-final-100-epochs"
log_paths = glob(os.path.join(exp_name, "*-model-logs-*"))
logs = [json_load(lp) for lp in log_paths]
print("logs:", len(logs))

In [ ]:
stat = "mean"
keys = list(logs[0][0].keys())
print(keys)

# Tensor n folds x n timessteps x n attributes
X = np.zeros((len(logs), len(logs[0]), len(logs[0][0])))
for i, log in enumerate(logs):
    for t, record in enumerate(log):
        for k, v in record.items():
            a = keys.index(k)
            value = v[stat]
            #print(i, t, a, k, value)
            X[i, t, a] = value        

In [ ]:
# take mean across the folds
import matplotlib.font_manager as font_manager
font_manager._rebuild()

import matplotlib.ticker as plticker
import matplotlib.font_manager as font_manager
loc = plticker.MultipleLocator(base=10.0) # this locator puts ticks at regular intervals
fontfamily = 'Arial'
font = {'fontname':fontfamily, 'fontsize': 10}
lg_font = font_manager.FontProperties(family=fontfamily, style='normal', size=8)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(5.5, 2.5*2), sharex=True)

# accuracy
axes[0].plot(X[:, :, keys.index('train_acc')].mean(axis=0), label="train")
axes[0].plot(X[:, :, keys.index('test_acc')].mean(axis=0), label="val")
axes[0].set_ylabel("Accuracy", **font)
axes[0].xaxis.set_major_locator(loc)
axes[0].legend(prop=lg_font)

# IoU
axes[1].plot(X[:, :, keys.index('train_iou')].mean(axis=0), label="train")
axes[1].plot(X[:, :, keys.index('test_iou')].mean(axis=0), label="val")
axes[1].set_ylabel("IoU", **font)
axes[1].xaxis.set_major_locator(loc)
axes[1].legend(prop=lg_font)

# common labels
axes[1].set_xlabel("Epoch", **font)

# save 
plt.savefig("t1-final/cv.tif", dpi=300)